# Introduction
## Goal. 
The goal of this lab is to study the application of Genetic Programming (GP) to various kinds of problems. We will also investigate the parametrization of the algorithm and its effect on the algorithmic performance. This lab continues the use of the deap $^{[1]}$ framework for the Python programming language we have seen in the previous lab. All the exercises are based on examples taken from the deap tutorial on Genetic Programming $^{[2]}$. If you did not participate in the previous lab, you may want to look that over first and then start this lab’s exercises.

--- 

[1]: Distributed Evolutionary Algorithms in Python: https://github.com/DEAP/deap

[2]: GP tutorials: https://deap.readthedocs.io/en/master/tutorials/advanced/gp.html

# Exercise 1
In this exercise we will use GP as a supervised Machine Learning (ML) technique for solving one of the classical problems where GP has shown the most straightforward and successful applications: symbolic regression. Symbolic regression is a form of function approximation that consists in fitting some data in the form of input-output samples to determine an expression (a symbolic formula) that approximates those samples. 
Assuming a set $S$ of $N$ samples $(S = {s_1, s_2, . . . , s_N })$, where each sample is composed of an n-dimensional input and a scalar output $y$, in the following form:

<center>$s_1 : [x_1, x_2, . . . , x_n] → y_1$
<center>$s_2 : [x_1, x_2, . . . , x_n] → y_2$
<center>$· · ·$
<center>$s_N : [x_1, x_2, . . . , x_n] → y_N$

the goal is to find an expression $f(·)$ such that $y = f(x_1, x_2, . . . , x_n) \forall s \in S$. Usually, an error metric such as the MSE (Mean Squared Error), $MSE= \frac{1}{N}\sum_{i \in N}[y - f(x_1, x_2, . . ., x_n)]^2$, is used to measure an individual’s fitness in terms of approximation error.

Here, we will consider a 1-dimensional input ($n = 1$), such that the set of samples is composed of couples $(x, y)$. By default, $N = 20$ equidistant samples are generated in the range $[−1, 1]$ by means of the polynomial function: 
    
<center> $y = f^{*}(x) = x^4 + x^3 + x^2 + x$

These samples are then used to evaluate the fitness of the evolving GP trees. The goal is to find an optimal tree describing a formula that approximates $f^{*}(·)$ such that the MSE is minimized$^{[1]}$.
The next cell contains the main code. Take a look at it and spend some time to understand its main steps.

One of the most important aspects is the definition of the *primitive set* (also called  non-terminal or function set), which in this case contains the four basic arithmetic operations, the trigonometric functions `cos` and `sin`, and the `neg` operator (such that `neg(x)=-x`). Note that the standard division operator is replaced by a custom function named `protectedDiv`, which handles divisions by zero without returning errors (this detail is often very important in GP-based symbolic regression, where it's not possible to check a priori all possible inputs to a division node). Note also that in `deap` the configuration of the Evolutionary Algorithm (in this case, Genetic Programming) is obtained by means of a series of function hooks registered in a `creator` object (that handles the initialization of the individual genotypes with their fitness values) and a `toolbox` object (that handles all the other methods that are called during the algorithm execution: evaluation, selection, mutation, crossover, etc.). All function hooks come with a user-defined alias (the first argument of the methods `create`, `register` and `decorate`). The evaluation of the fitness function (in this case, the MSE) is implemented in the method `evalSymbReg`, that calls the method `generatorFunction` (the only part of code you should change if you want to test a different generator function).

![tree.png](img/tree.png)

The relevant parameters of the Genetic Programming algorithm can be found at the beginning of the script. In particular, consider the parameters `GP_POP_SIZE`, `GP_NGEN`, `GP_CXPB`, `GP_MUTPB`,`GP_TRNMT_SIZE`, and `GP_HOF_SIZE`. These parameters represent, respectively: the population size (`GP_POP_SIZE`),  the number of generations (`GP_NGEN`), the crossover probability (`GP_CXPB`), the mutation probability (`GP_MUTPB`), the tournament size (`GP_TRNMT_SIZE`) and the size of the Hall-of-Fame (`GP_HOF_SIZE`)$^{[2]}$. See the documentation$^{[3]}$ for further explanations on the implementation details of this exercise.

At the end of the run, the script will show you (and save on files, in the `results` folder) a graphical representation$^{[4]}$ of the best evolved tree (similar to that shown in the figure above), as well as a comparison between the values of the real and GP-approximated data, and the evolutionary trends of the trees' fitness and size (i.e., number of nodes) across generations, see the figure below.

![tree.png](img/symbreg_merged.png)
    
- Is the GP algorithm able to approximate the given polynomial, with the standard configuration? What happens when you run the script multiple times? Do you always obtain the same results, or not? Why?
    
- Try to change the generator function (e.g. to include trigonometric functions) defined in the method `generatorFunction`. Is the GP algorithm able to approximate more complicated generator functions? Which parameters can you change to improve the results?

---
    
[1]: Note that while in this synthetic example we know that samples are generated by a certain \emph{generator function} $f^*(\cdot)$, in practical applications we don't know if this function even exists -or has a meaningful physical/mathematical formulation. In other words, the goal of GP is to approximate the mapping between inputs and outputs in a purely data-driven approach.
    
[2]: Note that *deap* uses a structure called Hall-of-Fame to store the best individual(s) found during the evolutionary process. The size of this archive is determined by the parameter `GP_HOF_SIZE`. This structure is only used to report the best solutions at the end of the evolutionary process, and it is not to be confused with the Hall-of-Fame used in competitive co-evolution.
    
[3]: See https://deap.readthedocs.io/en/master/examples/gp_symbreg.html
    
[4]: The Python packages `pygraphviz` and `networkx` are required for this.

In [ ]:
# Note: this cell will install the dependencies pygraphviz and networkx
# If you are using Windows, please follow the instructions that you will find here:
#       https://pygraphviz.github.io/documentation/stable/install.html#windows
# If you are still not able to import pygraphviz due to an error like:
# 'ImportError: DLL load failed while importing _graphviz'
# you may solve this error by adding e.g. "C:\Program Files\Graphviz\bin" to the PATH environment variable.
# !python3 -m pip install networkx
#!sudo apt-get install graphviz graphviz-dev
# !pip install pygraphviz

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

print(os.getcwd())

In [ ]:
import operator
import math
import random

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp
import os
from utils.plot_utils import plotTree
import matplotlib.pyplot as plt

"""
-------------------------------------------------------------------------
Edit this part to do the exercises
"""

GP_POP_SIZE = 300  # population size for GP
GP_NGEN = 40  # number of generations for GP
GP_CXPB, GP_MUTPB = 0.5, 0.1  # crossover and mutation probability for GP
GP_TRNMT_SIZE = 3  # tournament size for GP
GP_HOF_SIZE = 1  # size of the Hall-of-Fame for GP
seed = 0

"""
-------------------------------------------------------------------------
"""
folder = "results"


# Define new functions
def protectedDiv(left: float, right: float) -> float:
    try:
        return left / right
    except ZeroDivisionError:
        return 1


pset = gp.PrimitiveSet("MAIN", 1)

pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)

try:
    pset.addEphemeralConstant("rand101", lambda: random.randint(-1, 1))
except Exception:
    print("EphemeralConstant is already defined, if you changed it restart the kernel")
pset.renameArguments(ARG0="x")

try:
    del creator.FitnessMinSR  # type: ignore
    del creator.IndividualSR  # type: ignore
except Exception:
    pass
creator.create("FitnessMinSR", base.Fitness, weights=(-1.0,))
creator.create("IndividualSR", gp.PrimitiveTree, fitness=creator.FitnessMinSR)  # type: ignore

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.IndividualSR, toolbox.expr)  # type: ignore
toolbox.register("population", tools.initRepeat, list, toolbox.individual)  # type: ignore
toolbox.register("compile", gp.compile, pset=pset)


# TODO: try to change the expression e.g. to include trigonometric functions
def generatorFunction(x: float) -> float:
    # return math.sin(x)+math.cos(x)
    # return math.sin(x)*x**2
    # return math.sin(x)+5*x**2
    return x**4 + x**3 + x**2 + x


def evalSymbReg(individual: gp.PrimitiveTree, points: list[float]) -> tuple[float]:
    # Transform the tree expression in a callable function
    gpFunction = toolbox.compile(expr=individual)  # type: ignore
    # Evaluate the mean squared error between the expression and the real function
    sqerrors = ((gpFunction(x) - generatorFunction(x)) ** 2 for x in points)
    return (math.fsum(sqerrors) / len(points),)


toolbox.register("evaluate", evalSymbReg, points=[x / 10.0 for x in range(-10, 10)])
toolbox.register("select", tools.selTournament, tournsize=GP_TRNMT_SIZE)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)  # type: ignore

toolbox.decorate(
    "mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17)
)
toolbox.decorate(
    "mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17)
)

# --------------------------------------------------------------------

random.seed(seed)

pop = toolbox.population(n=GP_POP_SIZE)  # type: ignore
hof = tools.HallOfFame(GP_HOF_SIZE)

stats_fit = tools.Statistics(lambda ind: ind.fitness.values)  # type: ignore
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", numpy.mean)
mstats.register("std", numpy.std)
mstats.register("min", numpy.min)
mstats.register("max", numpy.max)

final_pop, logbook = algorithms.eaSimple(
    pop, toolbox, GP_CXPB, GP_MUTPB, GP_NGEN, stats=mstats, halloffame=hof, verbose=True
)

# --------------------------------------------------------------------

# plot GP tree

nodes, edges, labels = gp.graph(hof[0])
plotTree(nodes, edges, labels, "symbreg_" + str(seed), "results")  # type: ignore

# --------------------------------------------------------------------

# plot fitness vs size trends

gen = logbook.select("gen")
fit_mins = logbook.chapters["fitness"].select("min")
size_avgs = logbook.chapters["size"].select("avg")

fig = plt.figure("GP (fitness and tree size trend)")
ax1 = fig.add_subplot(111)
line1 = ax1.plot(gen, fit_mins, "b-", label="Minimum Fitness")
ax1.set_xlabel("Generation")
ax1.set_ylabel("Fitness", color="b")
for tl in ax1.get_yticklabels():
    tl.set_color("b")

ax2 = ax1.twinx()
line2 = ax2.plot(gen, size_avgs, "r-", label="Average Size")  # type: ignore
ax2.set_ylabel("Size", color="r")
for tl in ax2.get_yticklabels():
    tl.set_color("r")

lns = line1 + line2
labs = [ln.get_label() for ln in lns]
ax2.legend(lns, labs, loc=0)  # type: ignore

if folder is not None and not os.path.exists(folder):  # type: ignore
    os.makedirs(folder)
plt.savefig(folder + "/" + "trends_symbreg.png")

# plot real vs approximated values
x_ = points = [x / 10.0 for x in range(-100, 100)]
y_real = [generatorFunction(x) for x in x_]
gpFunction = toolbox.compile(expr=hof[0])  # type: ignore
y_gp = [gpFunction(x) for x in x_]
fig = plt.figure("GP (real vs approximated values)")
ax1 = fig.add_subplot(111)
line1 = ax1.plot(x_, y_real, label="Real")
line2 = ax1.plot(x_, y_gp, label="GP")
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.legend()
# --------------------------------------------------------------------

print("Best individual GP is %s, %s" % (hof[0], hof[0].fitness.values))

# Exercise 2

In this exercise we will use GP for solving different kinds of problems. In the next cells you will find various other examples of GP applications. 
Through these examples, we will see how GP can be flexibly applied to solve a broad range of problems beyond function approximation, such as classification and evolution of computer programs, with different features and levels of complexity. In particular, consider the following four cells:


 1. exercise parity: the even parity problem$^{[1]}$. This is a classical benchmark problem used in the GP literature: the goal is to _evolve a Boolean expression_ that produces the correct value of the even parity bit, given as input an array of $n$ bits. For instance, for an input size $n=6$ (Parity-6 problem), the goal is to match the parity bit value for each of the $2^6 = 64$ possible combinations of 6 bits$^{[2]}$. Thus the fitness of an individual (i.e., a candidate Boolean expression) can be calculated simply as the number of successful cases (to be maximized), such that the maximum attainable fitness is 64 in the case of 6 bits inputs. Note that since in this case all inputs/outputs are bits (Boolean values), the primitive set contains only Boolean operators (`and`, `or`, `xor`, `not`).
	
 2. exercise multiplexer: the multiplexer problem$^{[3]}$ is another classical benchmark problem used in the GP literature. Similarly to the even parity problem, this problem deals with Boolean values: the goal is _evolve a Boolean expression_ that is capable to reproduce the behavior of an electronic multiplexer (mux)$^{[4]}$. In the default configuration, a 3-8 multiplexer is used (3 select entries, labeled from A0 to A2, and 8 data entries, from D0 to D7), for a total number of $2^3\times2^8=2048$ possible combinations. Thus the fitness is the number of correct outputs over all $2048$ cases, and its maximum value is $2048$.
	
 3. exercise spambase: the ''spam base'' problem$^{[5]}$ consists in \emph{evolving a classifier} that is able to distinguish as spam/non-spam a given database of emails	(saved in `utils/utils_11/data/spambase.csv`). The database consists of $4601$ _labeled_ samples (as such, GP is used here as a supervised ML algorithm), each corresponding to a different email and comprising $57$ mixed integer/real-valued features describing various lexicographic/syntactic characteristics of the email$^{[6]}$, in addition to a binary label indicating if the email is spam (1) or not (0). The evolved classifier must then return a Boolean value which must be `True` if the email is spam, `False` otherwise. To avoid overfitting, each tree generated during the GP run is evaluated on a set of $400$ samples randomly selected from the database (so that the maximum achievable fitness, i.e. number of correct classifications, is $400$). Note that since data of different kinds must be manipulated by the classifier (float and Boolean values), in this case the GP algorithm is configured to handle `strongly typed` operations, i.e. for each function in the primitive set the type of inputs and outputs is specified (e.g. to avoid calling a Boolean function with float	values as inputs).
	
 4. exercise ant: the ''artificial ant'' problem$^{[7]}$ consists in _evolving a program_ that can successfully control an artificial ant so that it can find and ''eat'' all the food located in a certain environment (i.e., placed along a twisting trail on a square toroidal grid). The program can use three operations, `move_forward`, `turn_right` and `turn_left`, to move the ant forward one cell, turn right or turn left. Each of these operations takes one timestep. The sensing function `if_food_ahead` looks into the cell the ant is currently facing and then executes one of its two arguments, depending upon whether that cell contains food or is empty. Two additional custom operations, `prog2` and	`prog3` are provided to facilitate the evolution of more complex behaviors. These custom operations take two and three other operations as arguments, respectively, and execute them in sequence. The evaluation function uses an instance of a simple simulator to evaluate a given individual (i.e., a candidate program). Each individual is given $600$ timesteps to navigate a virtual map obtained from an external file (see the file `utils/utils_11/data/santafe_trail.txt`, where `#`, `.` and `S` indicate, respectively, a cell with food, an empty cell, and the starting cell). The fitness of each individual corresponds to the number of pieces of food picked up. In this example, the trail contains $89$ pieces of food in total. Therefore, an optimal individual would achieve a fitness of $89$.

Since these scripts have all the same structure as the one used in the first exercise (besides obviously the details of the specific problem, especially the evaluator function, and the definition of the primitive set), we won't go into the details of each of them. You can spend some time having a look at the code and trying to get the main steps of each of the four examples.

![mux_trend.png](img/es2_merged.png)

<center>Example fitness trends for the 3-8 multiplexer problem (left) and the ''artificial ant'' problem (right).


Try to run some experiments with one or more of these four scripts. As in the previous exercise, at the end of the run each script will show you (and save on files, in the `results` folder) a graphical representation of the best evolved tree, as well as the evolutionary trends of the trees' fitness across generations, see figures above.

 1. In the case of the multiplexer problem there is an additional parameter (`MUX_SELECT_LINES`) that allows you to scale the problem, increasing or decreasing its dimensionality. Similarly, for the parity problem there is a parameter (`PARITY_FANIN_M`) that allows you to change the problem dimensionality. This in turn will make the problem easier, or harder. If you tested one of these two problems, consider changing these parameters and observe the GP's behavior. Note, however, that these problems (especially the multiplexer) become computationally very complex when the dimensionality increases, so that experiments can be quite time-consuming!
    
 2. What kind of performance do you get, in general, on the tested problems? What happens when you change the parametrization of GP? 


---
[1]: Parity problem: https://deap.readthedocs.io/en/master/examples/gp_parity.html
    
[2]: Parity wikipedia: https://en.wikipedia.org/wiki/Parity_bit
    
[3]: Multiplexer problem: https://deap.readthedocs.io/en/master/examples/gp_multiplexer.html
    
[4]: See https://en.wikipedia.org/wiki/Multiplexer for more details
    
[5]: Spam base problem: https://deap.readthedocs.io/en/master/examples/gp_spambase.html
    
[6]: See http://archive.ics.uci.edu/ml/datasets/Spambase for a complete description of the features
    
[7]: Ant problem: https://deap.readthedocs.io/en/master/examples/gp_ant.html

# Parity problem

In [ ]:
import random
import operator

import numpy
from utils.plot_utils import plotTree, plotTrends
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

"""
-------------------------------------------------------------------------
Edit this part to do the exercises
"""

GP_POP_SIZE = 300  # population size for GP
GP_NGEN = 40  # number of generations for GP
GP_CXPB, GP_MUTPB = 0.5, 0.2  # crossover and mutation probability for GP
GP_TRNMT_SIZE = 3  # tournament size for GP
GP_HOF_SIZE = 1  # size of the Hall-of-Fame for GP

PARITY_FANIN_M = 6  # input size of the Parity problem
seed = 0
"""
-------------------------------------------------------------------------
"""

# Initialize Parity problem input and output matrices
PARITY_SIZE_M = 2**PARITY_FANIN_M

inputs: list[list[int]] = [[]] * PARITY_SIZE_M
outputs: list[int] = [0] * PARITY_SIZE_M

for i in range(PARITY_SIZE_M):
    inputs[i] = [0] * PARITY_FANIN_M
    value = i
    dividor = PARITY_SIZE_M
    parity = 1
    for j in range(PARITY_FANIN_M):
        dividor /= 2
        if value >= dividor:
            inputs[i][j] = 1
            parity = int(not parity)
            value -= dividor
        else:
            inputs[i][j] = 0
    outputs[i] = parity

pset = gp.PrimitiveSet("MAIN", PARITY_FANIN_M, "IN")
pset.addPrimitive(operator.and_, 2)
pset.addPrimitive(operator.or_, 2)
pset.addPrimitive(operator.xor, 2)
pset.addPrimitive(operator.not_, 1)
pset.addTerminal(1)
pset.addTerminal(0)

try:
    del creator.FitnessMaxParity  # type: ignore
    del creator.IndividualParity  # type: ignore
except Exception:
    pass
creator.create("FitnessMaxParity", base.Fitness, weights=(1.0,))
creator.create("IndividualParity", gp.PrimitiveTree, fitness=creator.FitnessMaxParity)  # type: ignore

toolbox = base.Toolbox()
toolbox.register("expr", gp.genFull, pset=pset, min_=3, max_=5)
toolbox.register(
    "individual",
    tools.initIterate,
    creator.IndividualParity,  # type: ignore
    toolbox.expr,  # type: ignore
)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)  # type: ignore
toolbox.register("compile", gp.compile, pset=pset)


def evalParity(individual: gp.PrimitiveTree) -> tuple[int]:
    func = toolbox.compile(expr=individual)  # type: ignore
    return (sum(func(*in_) == out for in_, out in zip(inputs, outputs)),)


toolbox.register("evaluate", evalParity)
toolbox.register("select", tools.selTournament, tournsize=GP_TRNMT_SIZE)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genGrow, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)  # type: ignore

# --------------------------------------------------------------------

random.seed(seed)

pop = toolbox.population(n=GP_POP_SIZE)  # type: ignore
hof = tools.HallOfFame(GP_HOF_SIZE)
stats = tools.Statistics(lambda ind: ind.fitness.values)  # type: ignore
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

final_pop, logbook = algorithms.eaSimple(
    pop, toolbox, GP_CXPB, GP_MUTPB, GP_NGEN, stats, halloffame=hof
)

# --------------------------------------------------------------------


nodes, edges, labels = gp.graph(hof[0])
plotTree(nodes, edges, labels, "parity" + "_" + str(seed), "results")

# plot fitness trends
plotTrends(logbook, "parity" + "_" + str(seed))

# --------------------------------------------------------------------

print("Best individual GP is %s, %s" % (hof[0], hof[0].fitness.values))

# Multiplexer problem

In [ ]:
import random
import operator
from typing import Any

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

from utils.plot_utils import plotTree, plotTrends

"""
-------------------------------------------------------------------------
Edit this part to do the exercises
"""

GP_POP_SIZE = 40  # population size for GP
GP_NGEN = 40  # number of generations for GP
GP_CXPB, GP_MUTPB = 0.8, 0.1  # crossover and mutation probability for GP
GP_TRNMT_SIZE = 7  # tournament size for GP
GP_HOF_SIZE = 1  # size of the Hall-of-Fame for GP

MUX_SELECT_LINES = 3  # number of select lines
seed = 0


"""
-------------------------------------------------------------------------
"""


def if_then_else(condition: bool, out1: Any, out2: Any) -> Any:
    return out1 if condition else out2


# Initialize Multiplexer problem input and output vectors
MUX_IN_LINES = 2**MUX_SELECT_LINES
MUX_TOTAL_LINES = MUX_SELECT_LINES + MUX_IN_LINES

# input : e.g. [A0 A1 A2 D0 D1 D2 D3 D4 D5 D6 D7] for a 8-3 mux
inputs = [[0] * MUX_TOTAL_LINES for _ in range(2**MUX_TOTAL_LINES)]
outputs = [None] * (2**MUX_TOTAL_LINES)

for i in range(2**MUX_TOTAL_LINES):
    value = i
    divisor = 2**MUX_TOTAL_LINES
    # Fill the input bits
    for j in range(MUX_TOTAL_LINES):
        divisor /= 2
        if value >= divisor:
            inputs[i][j] = 1
            value -= divisor

    # Determine the corresponding output
    indexOutput = MUX_SELECT_LINES
    for j, k in enumerate(inputs[i][:MUX_SELECT_LINES]):
        indexOutput += k * 2**j
    outputs[i] = inputs[i][indexOutput]

pset = gp.PrimitiveSet("MAIN", MUX_TOTAL_LINES, "IN")
pset.addPrimitive(operator.and_, 2)
pset.addPrimitive(operator.or_, 2)
pset.addPrimitive(operator.not_, 1)
pset.addPrimitive(if_then_else, 3)
pset.addTerminal(1)
pset.addTerminal(0)

try:
    del creator.FitnessMaxMux  # type: ignore
    del creator.IndividualMux  # type: ignore
except Exception:
    pass
creator.create("FitnessMaxMux", base.Fitness, weights=(1.0,))
creator.create("IndividualMux", gp.PrimitiveTree, fitness=creator.FitnessMaxMux)  # type: ignore

toolbox = base.Toolbox()
toolbox.register("expr", gp.genFull, pset=pset, min_=2, max_=4)
toolbox.register("individual", tools.initIterate, creator.IndividualMux, toolbox.expr)  # type: ignore
toolbox.register("population", tools.initRepeat, list, toolbox.individual)  # type: ignore
toolbox.register("compile", gp.compile, pset=pset)


def evalMultiplexer(individual: gp.PrimitiveTree) -> tuple[int]:
    func = toolbox.compile(expr=individual)  # type: ignore
    return (sum(func(*in_) == out for in_, out in zip(inputs, outputs)),)


toolbox.register("evaluate", evalMultiplexer)
toolbox.register("select", tools.selTournament, tournsize=GP_TRNMT_SIZE)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genGrow, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)  # type: ignore

# --------------------------------------------------------------------

random.seed(seed)

pop = toolbox.population(n=GP_POP_SIZE)  # type: ignore
hof = tools.HallOfFame(GP_HOF_SIZE)
stats = tools.Statistics(lambda ind: ind.fitness.values)  # type: ignore
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

final_pop, logbook = algorithms.eaSimple(
    pop, toolbox, GP_CXPB, GP_MUTPB, GP_NGEN, stats, halloffame=hof
)

# --------------------------------------------------------------------

# plot GP tree
nodes, edges, labels = gp.graph(hof[0])
plotTree(nodes, edges, labels, "multiplexer_" + str(seed), "results")

# plot fitness trends
plotTrends(logbook, "multiplexer_" + str(seed))

# --------------------------------------------------------------------

print("Best individual GP is %s, %s" % (hof[0], hof[0].fitness.values))

# Spam Base problem

In [ ]:
import random
import operator
import csv
import itertools

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

from utils.plot_utils import plotTree, plotTrends

"""
-------------------------------------------------------------------------
Edit this part to do the exercises
"""

GP_POP_SIZE = 100  # population size for GP
GP_NGEN = 40  # number of generations for GP
GP_CXPB, GP_MUTPB = 0.5, 0.2  # crossover and mutation probability for GP
GP_TRNMT_SIZE = 3  # tournament size for GP
GP_HOF_SIZE = 1  # size of the Hall-of-Fame for GP
seed = 0
"""
-------------------------------------------------------------------------
"""

# Read the spam list features and put it in a list of lists.
# The dataset is from http://archive.ics.uci.edu/ml/datasets/Spambase
# This example is a copy of the OpenBEAGLE example :
# http://beagle.gel.ulaval.ca/refmanual/beagle/html/d2/dbe/group__Spambase.html
with open("utils/utils_11/data/spambase.csv") as spambase:
    spamReader = csv.reader(spambase)
    spam = list(list(float(elem) for elem in row) for row in spamReader)

# defined a new primitive set for strongly typed GP
pset = gp.PrimitiveSetTyped("MAIN", itertools.repeat(float, 57), bool, "IN")

# boolean operators
pset.addPrimitive(operator.and_, [bool, bool], bool)
pset.addPrimitive(operator.or_, [bool, bool], bool)
pset.addPrimitive(operator.not_, [bool], bool)


# floating point operators
# Define a protected division function
def protectedDiv(left: float, right: float) -> float:
    try:
        return left / right
    except ZeroDivisionError:
        return 1


pset.addPrimitive(operator.add, [float, float], float)
pset.addPrimitive(operator.sub, [float, float], float)
pset.addPrimitive(operator.mul, [float, float], float)
pset.addPrimitive(protectedDiv, [float, float], float)


# logic operators
# Define a new if-then-else function
def if_then_else(input: bool, output1: Any, output2: Any) -> Any:
    if input:
        return output1
    else:
        return output2


pset.addPrimitive(operator.lt, [float, float], bool)
pset.addPrimitive(operator.eq, [float, float], bool)
pset.addPrimitive(if_then_else, [bool, float, float], float)

# terminals
try:
    pset.addEphemeralConstant("rand100", lambda: random.random() * 100, float)
except Exception:
    print("EphemeralConstant is already defined, if you changed it restart the kernel")
pset.addTerminal(False, bool)
pset.addTerminal(True, bool)

try:
    del creator.FitnessMaxSpam  # type: ignore
    del creator.IndividualSpam  # type: ignore
except Exception:
    pass
creator.create("FitnessMaxSpam", base.Fitness, weights=(1.0,))
creator.create("IndividualSpam", gp.PrimitiveTree, fitness=creator.FitnessMaxSpam)  # type: ignore

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.IndividualSpam, toolbox.expr)  # type: ignore
toolbox.register("population", tools.initRepeat, list, toolbox.individual)  # type: ignore
toolbox.register("compile", gp.compile, pset=pset)


def evalSpambase(individual: gp.PrimitiveTree) -> tuple[int]:
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)  # type: ignore
    # Randomly sample 400 mails in the spam database
    spam_samp = random.sample(spam, 400)
    # Evaluate the sum of correctly identified mail as spam
    result = sum(bool(func(*mail[:57])) is bool(mail[57]) for mail in spam_samp)
    return (result,)


toolbox.register("evaluate", evalSpambase)
toolbox.register("select", tools.selTournament, tournsize=GP_TRNMT_SIZE)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)  # type: ignore

# --------------------------------------------------------------------

random.seed(seed)

pop = toolbox.population(n=GP_POP_SIZE)  # type: ignore
hof = tools.HallOfFame(GP_HOF_SIZE)
stats = tools.Statistics(lambda ind: ind.fitness.values)  # type: ignore
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

final_pop, logbook = algorithms.eaSimple(
    pop, toolbox, GP_CXPB, GP_MUTPB, GP_NGEN, stats, halloffame=hof
)

# --------------------------------------------------------------------

# plot GP tree
nodes, edges, labels = gp.graph(hof[0])
plotTree(nodes, edges, labels, "spambase_" + str(seed), "results")

# plot fitness trends
plotTrends(logbook, "spambase_" + str(seed))

# --------------------------------------------------------------------

print("Best individual GP is %s, %s" % (hof[0], hof[0].fitness.values))

# Ant problem

In [ ]:
import copy
import random

import numpy

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp
from utils import plot_utils

"""
-------------------------------------------------------------------------
Edit this part to do the exercises
"""

GP_POP_SIZE = 300  # population size for GP
GP_NGEN = 40  # number of generations for GP
GP_CXPB, GP_MUTPB = 0.5, 0.2  # crossover and mutation probability for GP
GP_TRNMT_SIZE = 7  # tournament size for GP
GP_HOF_SIZE = 1  # size of the Hall-of-Fame for GP
seed = 0
"""
-------------------------------------------------------------------------
"""


def progn(*args: gp.Primitive) -> None:
    for arg in args:
        arg()  # type: ignore


def prog2(out1: gp.Primitive, out2: gp.Primitive) -> gp.Primitive:
    return partial(progn, out1, out2)  # type: ignore


def prog3(out1: gp.Primitive, out2: gp.Primitive, out3: gp.Primitive) -> gp.Primitive:
    return partial(progn, out1, out2, out3)  # type: ignore


def if_then_else(condition: Any, out1: Any, out2: Any) -> Any:
    out1() if condition() else out2()


class AntSimulator(object):
    direction = ["north", "east", "south", "west"]
    dir_row = [1, 0, -1, 0]
    dir_col = [0, 1, 0, -1]

    def __init__(self, max_moves: int):
        self.max_moves = max_moves
        self.moves = 0
        self.eaten = 0
        self.routine = None

    def _reset(self):
        self.row = self.row_start
        self.col = self.col_start
        self.dir = 1
        self.moves = 0
        self.eaten = 0
        self.matrix_exc = copy.deepcopy(self.matrix)

    @property
    def position(self):
        return (self.row, self.col, self.direction[self.dir])

    def turn_left(self):
        if self.moves < self.max_moves:
            self.moves += 1
            self.dir = (self.dir - 1) % 4

    def turn_right(self):
        if self.moves < self.max_moves:
            self.moves += 1
            self.dir = (self.dir + 1) % 4

    def move_forward(self):
        if self.moves < self.max_moves:
            self.moves += 1
            self.row = (self.row + self.dir_row[self.dir]) % self.matrix_row
            self.col = (self.col + self.dir_col[self.dir]) % self.matrix_col
            if self.matrix_exc[self.row][self.col] == "food":
                self.eaten += 1
            self.matrix_exc[self.row][self.col] = "passed"

    def sense_food(self) -> bool:
        ahead_row = (self.row + self.dir_row[self.dir]) % self.matrix_row
        ahead_col = (self.col + self.dir_col[self.dir]) % self.matrix_col
        return self.matrix_exc[ahead_row][ahead_col] == "food"

    def if_food_ahead(self, out1: gp.Primitive, out2: gp.Primitive) -> gp.Primitive:
        return partial(if_then_else, self.sense_food, out1, out2)  # type: ignore

    def run(self, routine: gp.Primitive):
        self._reset()
        while self.moves < self.max_moves:
            routine()  # type: ignore

    def parse_matrix(self, matrix: list[list[str]]):
        self.matrix = list()
        for i, line in enumerate(matrix):
            self.matrix.append(list())
            for j, col in enumerate(line):
                if col == "#":
                    self.matrix[-1].append("food")
                elif col == ".":
                    self.matrix[-1].append("empty")
                elif col == "S":
                    self.matrix[-1].append("empty")
                    self.row_start = self.row = i
                    self.col_start = self.col = j
                    self.dir = 1
        self.matrix_row = len(self.matrix)
        self.matrix_col = len(self.matrix[0])
        self.matrix_exc = copy.deepcopy(self.matrix)


ant = AntSimulator(600)

pset = gp.PrimitiveSet("MAIN", 0)
pset.addPrimitive(ant.if_food_ahead, 2)
pset.addPrimitive(prog2, 2)
pset.addPrimitive(prog3, 3)
pset.addTerminal(ant.move_forward)
pset.addTerminal(ant.turn_left)
pset.addTerminal(ant.turn_right)

try:
    del creator.FitnessMaxAnt  # type: ignore
    del creator.IndividualAnt  # type: ignore
except Exception:
    pass
creator.create("FitnessMaxAnt", base.Fitness, weights=(1.0,))
creator.create("IndividualAnt", gp.PrimitiveTree, fitness=creator.FitnessMaxAnt)  # type: ignore

toolbox = base.Toolbox()

# Attribute generator
toolbox.register("expr_init", gp.genFull, pset=pset, min_=1, max_=2)

# Structure initializers
toolbox.register(
    "individual",
    tools.initIterate,
    creator.IndividualAnt,  # type: ignore
    toolbox.expr_init,  # type: ignore
)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)  # type: ignore


def evalArtificialAnt(individual: gp.PrimitiveTree) -> tuple[int]:
    # Transform the tree expression to functionnal Python code
    routine = gp.compile(individual, pset)
    # Run the generated routine
    ant.run(routine)
    return (ant.eaten,)


toolbox.register("evaluate", evalArtificialAnt)
toolbox.register("select", tools.selTournament, tournsize=GP_TRNMT_SIZE)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)  # type: ignore

# --------------------------------------------------------------------

random.seed(seed)

with open("utils/utils_11/data/santafe_trail.txt") as trail_file:
    ant.parse_matrix(trail_file)  # type: ignore

pop = toolbox.population(n=GP_POP_SIZE)  # type: ignore
hof = tools.HallOfFame(GP_HOF_SIZE)
stats = tools.Statistics(lambda ind: ind.fitness.values)  # type: ignore
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

final_pop, logbook = algorithms.eaSimple(
    pop, toolbox, GP_CXPB, GP_MUTPB, GP_NGEN, stats, halloffame=hof
)

# --------------------------------------------------------------------

# plot GP tree
nodes, edges, labels = gp.graph(hof[0])
plot_utils.plotTree(nodes, edges, labels, "ant_" + str(seed), "results")

# plot fitness trends
plot_utils.plotTrends(logbook, "ant_" + str(seed))

# --------------------------------------------------------------------

print("Best individual GP is %s, %s" % (hof[0], hof[0].fitness.values))

# Instructions and questions

Concisely note down your observations from the previous exercises (follow the bullet points) and think about the following questions. 

 - What are the main strengths and limitations of GP, in your opinion?
 
 - In which kind of applications do you think that GP could be more useful than other kinds of black-box Machine Learning techniques, such as Neural Network? Why?

# BONUS
In the enxt cell there is a variant of the first exercise. The main difference between this and the original exercise is that in this variant a _competitive co-evolutionary approach_ is used for solving the symbolic regression problem. Rather than determining _a priori_ the samples that are used for evolving the GP trees, this approach uses a Genetic Algorithm (GA) to _evolve_ a fix-sized set of samples that is fed to a tree evolved by GP. The goal of GP is to _minimize_ the MSE, while the goal of GA is to _maximize_ it, i.e. to generate a set of samples that produces the largest fitting error in the evolving GP trees. 

To do so, at the initial generation a random GA (GP) individual is sampled from the corresponding initial population and tested against all individuals in the initial population of GP (GA). Then at each subsequent generation the best GA individual (i.e., the set of samples that produced the largest MSE so far) is fed to all the trees in the current GP population. Vice versa, the best GP tree (i.e., the one that showed the lowest MSE so far) is fed with all the individuals (i.e., sets of samples) in the current GA population. This form of _arms race_ forces the GP trees to become more adaptive, avoiding overfitting to a particular set of samples, thus also becoming better at generalizing and extrapolating data outside the specific set of samples that was used for fitting. Note, however, that in some cases this approach may lead to excessively large trees that are even impossible to evaluate$^{[1]}$. Indeed, due to an implicit limitation of Python, trees deeper than 90 layers cannot be evaluated. In these cases the script gives the following error:
`MemoryError: DEAP : Error in tree evaluation : Python cannot evaluate a tree higher than 90. To avoid this problem, you should use bloat control on your operators. See the DEAP documentation for more information. DEAP will now abort`.

Run some tests with the co-evolutionary variant of the symbolic regression experiment. At the end of the evolutionary process, you should see a separate fitness trend for the GA and the GP evolution. 

 - Is this approach better (in terms of time to converge, approximation results, and robustness across multiple runs) than the one based on GP alone? Why?
 - Try to change the parameters of the co-evolutionary scheme to test how the system behaves under different configurations. Focus in particular on the population size used in the two algorithms (`GA_POP_SIZE` and `GP_POP_SIZE`), and the number of samples contained in each GA individual (`GA_REP_IND`). 



---
[1]: This is the typical effect of _bloat_. In the most extreme cases, this could even cause an internal Python `MemoryError`


In [ ]:
import os

import random
import operator
import math

import numpy

from deap import base
from deap import creator
from deap import tools
from deap import gp

import matplotlib.pyplot as plt
import utils.plot_utils as plot_utils

"""
-------------------------------------------------------------------------
Edit this part to do the exercises
"""

GA_POP_SIZE = 200  # population size for GA
NGEN = 50  # number of generations (for both GA and GP)
GA_TRNMT_SIZE = 3  # tournament size for GA
GA_REP_IND = 10  # number of arrays for each GA individual
GA_CXPB, GA_MUTPB = 0.5, 0.2  # crossover and mutation probability for GA


GP_POP_SIZE = 200  # population size for GP
GP_NGEN = 40  # number of generations for GP
GP_CXPB, GP_MUTPB = 0.5, 0.2  # crossover and mutation probability for GP
GP_TRNMT_SIZE = 3  # tournament size for GP
GP_HOF_SIZE = 1  # size of the Hall-of-Fame for GP

seed = 0
"""
-------------------------------------------------------------------------
"""

try:
    del creator.FitnessMaxGA  # type: ignore
    del creator.IndividualGA  # type: ignore
except Exception:
    pass
creator.create("FitnessMaxGA", base.Fitness, weights=(1.0,))
creator.create("IndividualGA", list, fitness=creator.FitnessMaxGA)  # type: ignore

toolbox_ga = base.Toolbox()

toolbox_ga.register("float", random.uniform, -1, 1)
toolbox_ga.register(
    "individual",
    tools.initRepeat,
    creator.IndividualGA,  # type: ignore
    toolbox_ga.float,  # type: ignore
    GA_REP_IND,  # type: ignore
)
toolbox_ga.register("population", tools.initRepeat, list, toolbox_ga.individual)  # type: ignore

toolbox_ga.register("select", tools.selTournament, tournsize=GA_TRNMT_SIZE)
toolbox_ga.register("mate", tools.cxTwoPoint)
# we leave the internal parameters of mutation (mu, sigma and indpb) as they are
toolbox_ga.register("mutate", tools.mutGaussian, mu=0, sigma=0.01, indpb=0.05)


def protectedDiv(left: float, right: float) -> float:
    try:
        return left / right
    except ZeroDivisionError:
        return 1


pset = gp.PrimitiveSet("MAIN", 1)

pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)

try:
    pset.addEphemeralConstant("rand101", lambda: random.randint(-1, 1))
except Exception:
    print("EphemeralConstant is already defined, if you changed it restart the kernel")
pset.renameArguments(ARG0="x")

try:
    del creator.FitnessMinGP  # type: ignore
    del creator.IndividualGP  # type: ignore
except Exception:
    pass
creator.create("FitnessMinGP", base.Fitness, weights=(-1.0,))
creator.create("IndividualGP", gp.PrimitiveTree, fitness=creator.FitnessMinGP)  # type: ignore

toolbox_gp = base.Toolbox()
toolbox_gp.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox_gp.register(
    "individual",
    tools.initIterate,
    creator.IndividualGP,  # type: ignore
    toolbox_gp.expr,  # type: ignore
)
toolbox_gp.register("population", tools.initRepeat, list, toolbox_gp.individual)  # type: ignore
toolbox_gp.register("compile", gp.compile, pset=pset)


# TODO: try to change the expression e.g. to include trigonometric functions
def generatorFunction(x: float) -> float:
    # return math.sin(x)+math.cos(x)
    # return math.sin(x)*x**2
    # return math.sin(x)+5*x**2
    return x**4 + x**3 + x**2 + x


def evalSymbReg(individual: gp.PrimitiveTree, points: list[float]) -> tuple[float]:
    # Transform the tree expression in a callable function
    gpFunction = toolbox_gp.compile(expr=individual)  # type: ignore
    # Evaluate the mean squared error between the expression and the real function
    sqerrors = ((gpFunction(x) - generatorFunction(x)) ** 2 for x in points)
    return (math.fsum(sqerrors) / len(points),)


toolbox_gp.register("evaluate", evalSymbReg, points=[x / 10.0 for x in range(-10, 10)])
toolbox_gp.register("select", tools.selTournament, tournsize=GP_TRNMT_SIZE)
toolbox_gp.register("mate", gp.cxOnePoint)
toolbox_gp.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox_gp.register("mutate", gp.mutUniform, expr=toolbox_gp.expr_mut, pset=pset)  # type: ignore

toolbox_gp.decorate(
    "mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17)
)
toolbox_gp.decorate(
    "mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17)
)

# --------------------------------------------------------------------
random.seed(seed)

pop_ga = toolbox_ga.population(n=GA_POP_SIZE)  # type: ignore
pop_gp = toolbox_gp.population(n=GP_POP_SIZE)  # type: ignore

stats = tools.Statistics(lambda ind: ind.fitness.values)  # type: ignore
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

logbook = tools.Logbook()
logbook.header = "gen", "type", "evals", "std", "min", "avg", "max"  # type: ignore

best_ga = tools.selRandom(pop_ga, 1)[0]
best_gp = tools.selRandom(pop_gp, 1)[0]

for ind in pop_gp:
    ind.fitness.values = toolbox_gp.evaluate(ind, points=best_ga)  # type: ignore

for ind in pop_ga:
    ind.fitness.values = toolbox_gp.evaluate(best_gp, points=ind)  # type: ignore

record = stats.compile(pop_ga)
logbook.record(gen=0, type="ga", evals=len(pop_ga), **record)

record = stats.compile(pop_gp)
logbook.record(gen=0, type="gp", evals=len(pop_gp), **record)

print(logbook.stream)

# Begin the evolution
for g in range(1, NGEN):
    # Select and clone the offspring
    off_ga = toolbox_ga.select(pop_ga, len(pop_ga))  # type: ignore
    off_gp = toolbox_gp.select(pop_gp, len(pop_gp))  # type: ignore
    off_ga = [toolbox_ga.clone(ind) for ind in off_ga]  # type: ignore
    off_gp = [toolbox_gp.clone(ind) for ind in off_gp]  # type: ignore

    # Apply crossover and mutation
    for ind1, ind2 in zip(off_ga[::2], off_ga[1::2]):
        if random.random() < GA_CXPB:
            toolbox_ga.mate(ind1, ind2)  # type: ignore
            del ind1.fitness.values
            del ind2.fitness.values

    for ind1, ind2 in zip(off_gp[::2], off_gp[1::2]):
        if random.random() < GP_CXPB:
            toolbox_gp.mate(ind1, ind2)  # type: ignore
            del ind1.fitness.values
            del ind2.fitness.values

    for ind in off_ga:
        if random.random() < GA_MUTPB:
            toolbox_ga.mutate(ind)  # type: ignore
            del ind.fitness.values

    for ind in off_gp:
        if random.random() < GP_MUTPB:
            toolbox_gp.mutate(ind)  # type: ignore
            del ind.fitness.values

    # Evaluate the individuals with an invalid fitness
    for ind in off_ga:
        ind.fitness.values = toolbox_gp.evaluate(best_gp, points=ind)  # type: ignore

    for ind in off_gp:
        ind.fitness.values = toolbox_gp.evaluate(ind, points=best_ga)  # type: ignore

    # Replace the old population by the offspring
    pop_ga = off_ga
    pop_gp = off_gp

    record = stats.compile(pop_ga)
    logbook.record(gen=g, type="ga", evals=len(pop_ga), **record)

    record = stats.compile(pop_gp)
    logbook.record(gen=g, type="gp", evals=len(pop_gp), **record)
    print(logbook.stream)

    best_ga = tools.selBest(pop_ga, 1)[0]
    best_gp = tools.selBest(pop_gp, 1)[0]

# --------------------------------------------------------------------

# plot GP tree

nodes, edges, labels = gp.graph(best_gp)

plot_utils.plotTree(nodes, edges, labels, "coev_" + str(seed), "results")

# --------------------------------------------------------------------

# plot fitness trends

folder = "results"
if folder is not None and not os.path.exists(folder):  # type: ignore
    os.makedirs(folder)
name = "coev_" + str(seed)

gen = numpy.array(logbook.select("gen"))
fit_type = numpy.array(logbook.select("type"))
fit_min = numpy.array(logbook.select("min"))
fit_max = numpy.array(logbook.select("max"))
fit_avg = numpy.array(logbook.select("avg"))
fit_std = numpy.array(logbook.select("std"))

ga_entries = numpy.array([i for i, val in enumerate(fit_type) if val == "ga"])
gp_entries = numpy.array([i for i, val in enumerate(fit_type) if val == "gp"])

fig = plt.figure("GA (fitness trend)")
ax1 = fig.add_subplot(111)
line1 = ax1.plot(gen[ga_entries], fit_min[ga_entries], label="Min")
line2 = ax1.plot(gen[ga_entries], fit_max[ga_entries], label="Max")
line3 = ax1.errorbar(
    gen[ga_entries], fit_avg[ga_entries], yerr=fit_std[ga_entries], label="Avg"
)
ax1.set_xlabel("Generation")
ax1.set_ylabel("Fitness")
ax1.set_yscale("log")
ax1.set_xlim(0, len(gen[ga_entries]) - 1)
ax1.set_title("GA")
ax1.legend()
plt.savefig(folder + "/" + "trends_" + name + "_ga.png")

fig = plt.figure("GP (fitness trend)")
ax2 = fig.add_subplot(111)
line1 = ax2.plot(gen[gp_entries], fit_min[gp_entries], label="Min")
line2 = ax2.plot(gen[gp_entries], fit_max[gp_entries], label="Max")
line3 = ax2.errorbar(
    gen[gp_entries], fit_avg[gp_entries], yerr=fit_std[gp_entries], label="Avg"
)
ax2.set_xlabel("Generation")
ax2.set_ylabel("Fitness")
ax2.set_yscale("log")
ax2.set_xlim(0, len(gen[gp_entries]) - 1)
ax2.set_title("GP")
ax2.legend()
plt.savefig(folder + "/" + "trends_" + name + "_gp.png")

plt.show()

# plot real vs. GP-approximated values
x_ = points = [x / 10.0 for x in range(-100, 100)]
y_real = [generatorFunction(x) for x in x_]
gpFunction = toolbox_gp.compile(expr=best_gp)  # type: ignore
y_gp = [gpFunction(x) for x in x_]
fig = plt.figure("GP (real vs approximated values)")
ax1 = fig.add_subplot(111)
line1 = ax1.plot(x_, y_real, label="Real")
line2 = ax1.plot(x_, y_gp, label="GP")
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.legend()
plt.show()

# --------------------------------------------------------------------

print("Best individual GA is %s, %s" % (best_ga, best_ga.fitness.values))
print("Best individual GP is %s, %s" % (best_gp, best_gp.fitness.values))